In [2]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
import math
import scipy as sp

import warnings
warnings.filterwarnings("ignore")

In [12]:
def load() -> pd.DataFrame:
    """Loading dataset
    @return: dataset
    """
    dataset = pd.read_csv('housing.csv')
    # dataset = dataset.fillna(method='bfill')
    return dataset

def copy_df(df) -> pd.DataFrame:
    """copy df to new temp df
    @param df: user input dataset
    @return: new df
    """
    return df.copy()
# normalize dataset missing values with mean
def normalize_missing_values(dataset) -> pd.DataFrame:
    """normalize dataset missing values with mean for numerical columns
    @param dataset: user input dataset
    @return: normalized dataset
    """
    for col in dataset.columns:
        # print("column: ",col, "type: ", dataset[col].dtype)
        if dataset[col].dtype == 'object':
            dataset[col] = dataset[col].fillna(method='bfill')
        else:
            dataset[col] = dataset[col].fillna(dataset[col].mean()) 
    return dataset

def print_dataset(dataset) -> None:
    """print dataset inforamtion
    @param dataset: user input dataset
    @return: print dataset information via head(), info(), shape()
    """
    print(dataset.head())
    print(dataset.info())
    print(dataset.describe())
    print(dataset.columns)
    print(dataset.iloc[:, -1:].value_counts())

dataset = load()
# dataset = normalize_missing_values(dataset)
print_dataset(dataset)

   longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                41.0        880.0           129.0   
1    -122.22     37.86                21.0       7099.0          1106.0   
2    -122.24     37.85                52.0       1467.0           190.0   
3    -122.25     37.85                52.0       1274.0           235.0   
4    -122.25     37.85                52.0       1627.0           280.0   

   population  households  median_income  median_house_value ocean_proximity  
0       322.0       126.0         8.3252            452600.0        NEAR BAY  
1      2401.0      1138.0         8.3014            358500.0        NEAR BAY  
2       496.0       177.0         7.2574            352100.0        NEAR BAY  
3       558.0       219.0         5.6431            341300.0        NEAR BAY  
4       565.0       259.0         3.8462            342200.0        NEAR BAY  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639

In [5]:
def k_fold_train_test_split(data, k_fold_number):
    """
    Splits data into k folds and returns the training and testing sets for each fold.
    """
    x_train = []
    x_test = []
    y_train = []
    y_test = []
    for i in range(k_fold_number):
        temp_train = data.iloc[int(len(data) / k_fold_number * i):int(len(data) / k_fold_number * (i + 1)), :]
        temp_test = data.iloc[int(len(data) / k_fold_number * (i + 1)):int(len(data) / k_fold_number * (i + 2)), :]
        x_train.append(temp_train.drop(['result'], axis=1))
        x_test.append(temp_test.drop(['result'], axis=1))
        y_train.append(temp_train['result'])
        y_test.append(temp_test['result'])
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = k_fold_train_test_split(dataset, 5)

for index, value in enumerate(x_test):
    print(f" x_test[{index}] : {value.shape}")

KeyError: "['result'] not found in axis"